In [3]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import os

In [13]:
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [5]:
# Разрешаем доступ к гугл диску, в котором находится файл с данными
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [6]:
#  Импортируем файл с данными
candy = pd.read_csv('gdrive/MyDrive/Python/lab_1/Data.dat', sep=";", header = 0, index_col=False)


In [7]:
# Выводим первые 10 строк
candy.head(10)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,4,5,5,5,3,3,3,3,3,3,3
1,5,4,5,5,3,3,3,3,3,3,3
2,5,5,4,5,3,3,3,3,3,3,3
3,5,4,5,5,3,3,3,3,3,3,3
4,4,5,5,5,3,3,3,3,3,3,3
5,5,4,5,5,3,3,3,3,3,3,3
6,5,5,4,5,3,3,3,3,3,3,3
7,5,4,5,5,3,3,3,3,3,3,3
8,5,5,5,5,3,3,3,3,3,3,3
9,5,4,5,4,3,3,3,3,3,3,3


In [16]:
dbscan = DBSCAN(eps=2, metric='euclidean', min_samples=5)
#  Обучим модель  DBSCAN
dbscan.fit(candy)
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ -1   7]
 [  0 110]
 [  1 101]
 [  2  61]
 [  3  59]]


Первая попытка и достаточно неплохой результат!


In [17]:
dbscan = DBSCAN(eps=3, metric='euclidean', min_samples=5)
dbscan.fit(candy)
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 110]
 [  1 108]
 [  2 120]]


In [18]:
dbscan = DBSCAN(eps=1, metric='euclidean', min_samples=5)
dbscan.fit(candy)
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ -1  44]
 [  0 101]
 [  1  49]
 [  2  15]
 [  3  10]
 [  4   6]
 [  5  37]
 [  6  15]
 [  7   9]
 [  8  52]]


Тааак, значит epsilon трогать нельзя, попробуем поменять min_samples

In [19]:
dbscan = DBSCAN(eps=2, metric='euclidean', min_samples=50)
dbscan.fit(candy)
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ -1   7]
 [  0 110]
 [  1 101]
 [  2  61]
 [  3  59]]


Вспоминая результат иерархического и k-means методов, можно сделать вывод, что 7 объектов из 1-ого кластера метод DBScan посчитал выбросами. Давайте это проверим, сверив результат этого метода с итерационным.

In [30]:
# Проводим иерархический анализ
link = linkage(candy, 'ward', 'euclidean')

# Записываем результаты кластеризации в таблицу candy
candy['cluster'] = fcluster(link, 4, criterion='maxclust')
candy['dbscan'] = dbscan.labels_

candy.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,cluster,dbscan
0,4,5,5,5,3,3,3,3,3,3,3,1,0
1,5,4,5,5,3,3,3,3,3,3,3,1,0
2,5,5,4,5,3,3,3,3,3,3,3,1,0
3,5,4,5,5,3,3,3,3,3,3,3,1,0
4,4,5,5,5,3,3,3,3,3,3,3,1,0


In [31]:
rez = pd.crosstab(candy['dbscan'], candy['cluster'])
print(rez)

cluster    1    2   3   4
dbscan                   
-1         0    7   0   0
 0       110    0   0   0
 1         0  101   0   0
 2         0    0   0  61
 3         0    0  59   0


Видно, что все класстеры совпадают, кроме 2ого кластера, в который иерархический метод занес 108 объектов, а DBScan разбил на 101 (в 1-ом кластере) и 7 выбросов (в -1 кластере)